In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import hyperspy.api as hs

In [ ]:
path = "../../../Masteroppgave/2023-03-08_EDS-Apreo/exports/"
s = hs.load(path + 'GaAs_30kV_50pA.emsa', signal_type='EDS_SEM')
elements = ['Ga', 'As']
lines = ['Ga_Ll','Ga_La', 'As_La',  'Ga_Ka', 'Ga_Kb', 'As_Ka', 'As_Kb']
names = ['Ga Ll','Ga Lα', 'As Lα', 'Ga Kα', 'Ga Kβ',  'As Kα', 'As Kβ']
s.set_elements(elements)
s.data[:33] = 0
m = s.create_model()
m.fit()
scale = s.axes_manager[0].scale
offset = s.axes_manager[0].offset
channels = np.arange(0, 2048)
kev = s.axes_manager[0].axis
x = channels
x = kev

In [ ]:
def kev2channel(kev):
    return (kev - offset) / scale
def channel2kev(channel):
    return channel * scale + offset
def remove_low_values(y, threshold):
    y[y < threshold] = np.nan
    return y
def theoretical_energy(line, what='energy (keV)'): # what can be 'energy (keV)' or 'weight'
    return hs.material.elements[line.split('_')[0]]['Atomic_properties']['Xray_lines'][line.split('_')[1]][what]

In [ ]:
# fig = go.Figure()
# fig.update_layout(font=dict(family="EB Garamond SemiBold", size=16, color="black"), margin=dict(l=30, r=30, t=30, b=50))
# fig.update_layout(width=800,height=400,legend=dict(yanchor="top",y=0.99,xanchor="right",x=0.99))


# for l in lines:
#     y = m[l].function(kev) * scale + m[0].function(kev) * scale
#     y = remove_low_values(y, 100)
#     fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name=f'{l}'))# {round(theoretical_energy(l),2):>10}'))

# fig.add_trace(go.Scatter(x=x, y=s.data, mode='lines', name='data', visible='legendonly'))
# fig.add_trace(go.Scatter(x=x, y=m.as_signal().data, mode='lines', name='fit'))


# fig.update_xaxes(title_text="Energy [keV]", range=[-.2, 14.5])#, showticklabels=False)
# fig.update_yaxes(title_text="Intensity [counts]", range=[-2500, 55e3])#, showticklabels=False)

# # log y
# fig.update_yaxes(type="log", range=[2, 5])

# # annotations
# # annotate E1-E(Si Ka) at 751 3000
# # fig.add_annotation(text="E<sub>1</sub>", x=130, y=47000, showarrow=False)
# # fig.add_annotation(text="Si Kα", x=215, y=6900, showarrow=False)


# tickvals = [kev2channel(kev) for kev in [-.2, 2, 4, 6, 8, 10, 12]]
# ticktext = [str(round(channel2kev(tickval),2)) for tickval in tickvals]
# # fig.update_xaxes(tickvals=tickvals, ticktext=ticktext)

# fig

In [ ]:
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.08) #, row_heights=[0.2, 0.8])
fig.update_layout(font=dict(family="EB Garamond SemiBold", size=16, color="black"), margin=dict(l=0, r=10, t=10, b=10))
fig.update_layout(width=800,height=600,legend=dict(yanchor="top",y=1.0,xanchor="right",x=1.22))

fig.update_xaxes(range=[-.2, 14.5])
fig.update_yaxes(range=[-2500, 55e3])
fig.update_xaxes(title_text="Energy [keV]", row=3, col=1)
fig.update_yaxes(type="log", range=[2, 5])

fig.update_yaxes(title_text="(a)", row=1, col=1, title_standoff=0)
fig.update_yaxes(title_text='Intensity [counts]<br>(b)', row=2, col=1, title_standoff=19)
fig.update_yaxes(title_text="(c)", row=3, col=1, title_standoff=0)

tickvals_y = [100, 1000, 10000, 100000]
ticktext_y = ['0.1k', '1k', '10k', '100k']

fig.update_yaxes(tickvals=tickvals_y, ticktext=ticktext_y)

colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52'] # plotly colors

# ROW 1
for l in lines:
    e = theoretical_energy(l)
    w = m[l].function(e) * scale #+ m[0].function(e) * scale
    fig.add_trace(go.Scatter(x=[e, e], y=[0, w], mode='lines', name=f'{names[lines.index(l)]}', line_color=colors[lines.index(l)], line_width=3), row=1, col=1)

# ROW 2
for l in lines:
    buff = 0.4
    x_range = np.arange(theoretical_energy(l) - buff, theoretical_energy(l) + buff, 0.01)
    y = m[l].function(x_range) * scale + m[0].function(x_range) * scale
    # y = m[l].function(x_range) * scale #+ m[0].function(x_range) * scale
    # y = remove_low_values(y)
    fig.add_trace(go.Scatter(x=x_range, y=y, mode='lines', line_color=colors[lines.index(l)], showlegend=False), row=2, col=1)# {round(theoretical_energy(l),2):>10}'))
fig.add_trace(go.Scatter(x=x, y=m[0].function(x) * scale, mode='lines', name='Background', line_color=colors[-1]), row=2, col=1)

# ROW 3
# fig.add_trace(go.Scatter(x=x, y=m.as_signal().data, mode='lines', name='Fit', line_color=colors[-3]), row=3, col=1)
fig.add_trace(go.Scatter(x=x, y=s.data, mode='lines', name='Real data', line_color=colors[-2]), row=3, col=1)

a_text = names 
a_x = [0.42, 0.51, 1.9,   8.5, 9.9, 11.2, 12.3]
a_y = [3.5,   4.7, 4.2,   4.3, 3.55,  4.1, 3.2]
for i, t in enumerate(a_text):
    fig.add_annotation(text=t, x=a_x[i], y=a_y[i], showarrow=False, row=1, col=1, xref='paper', yref='paper', font=dict(size=14, color=colors[i]))
    # fig.add_annotation(text=t, x=a_x[i], y=a_y[i], showarrow=False, row=3, col=1, xref='paper', yref='paper', font=dict(size=14))

    # fig.add_annotation(text=t, x=a_x[i], y=a_y[i], showarrow=False, row=2, col=1, xref='paper', yref='paper', font=dict(size=14, color=colors[i]))


fig

In [ ]:
fig.write_image("../figures/theoretical2realSpectrum.pdf")
fig.write_image('../../mastersthesis/figures/eds_theoretical2realSpectrum.pdf')